# Preliminaries

In [1]:
# install sent2vec
!pip install git+https://github.com/epfml/sent2vec

  Cloning https://github.com/epfml/sent2vec to /tmp/pip-req-build-klkg4fel
  Running command git clone -q https://github.com/epfml/sent2vec /tmp/pip-req-build-klkg4fel
  Created wheel for sent2vec: filename=sent2vec-0.0.0-cp36-cp36m-linux_x86_64.whl size=1137401 sha256=80df24acfedcfa42c7400a4403d632d1740b391296d814505a5f30d86c7772dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-k5nsuqff/wheels/f5/1a/52/b5f36e8120688b3f026ac0cefe9c6544905753c51d8190ff17
Successfully built sent2vec


Write requirements to file, anytime you run it, in case you have to go back and recover dependencies.

Latest known such requirements are hosted for each notebook in the companion github repo, and can be pulled down and installed here if needed. Companion github repo is located at https://github.com/azunre/transfer-learning-for-nlp

In [2]:
!pip freeze > kaggle_image_requirements.txt

# Download IMDB Movie Review Dataset
Download IMDB dataset

In [3]:
import random
import pandas as pd

## Read-in the reviews and print some basic descriptions of them

!wget -q "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
!tar xzf aclImdb_v1.tar.gz

wget: /opt/conda/lib/libuuid.so.1: no version information available (required by wget)


# Define Tokenization, Stop-word and Punctuation Removal Functions
Before proceeding, we must decide how many samples to draw from each class. We must also decide the maximum number of tokens per email, and the maximum length of each token. This is done by setting the following overarching hyperparameters

**Preliminary overarching hyperparameters**


In [4]:
Nsamp = 1000 # number of samples to generate in each class - 'positive', 'negative'
maxtokens = 200 # the maximum number of tokens per document
maxtokenlen = 100 # the maximum length of each token

**Tokenization**

In [5]:
def tokenize(row):
    if row is None or row is '':
        tokens = ""
    else:
        tokens = str(row).split(" ")[:maxtokens]
    return tokens

**Use regular expressions to remove unnecessary characters**

Next, we define a function to remove punctuation marks and other nonword characters (using regular expressions) from the emails with the help of the ubiquitous python regex library. In the same step, we truncate all tokens to hyperparameter maxtokenlen defined above.

In [6]:
import re

def remove_reg_expressions(row):
    tokens = []
    try:
        for token in row:
            token = token.lower()
            token = re.sub(r'[\W\d]', " ", token)
            token = token[:maxtokenlen] # truncate token
            tokens.append(token)
    except:
        token = ""
        tokens.append(token)
    return tokens

**Stop-word removal**

Stop-words are also removed. Stop-words are words that are very common in text but offer no useful information that can be used to classify the text. Words such as is, and, the, are are examples of stop-words. The NLTK library contains a list of 127 English stop-words and can be used to filter our tokenized strings.

In [7]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')    

# these stopwords may indicate positivity/negativity of sentiment, so we remove them (keep them in corpus)
# stopwords.remove("no")
# stopwords.remove("nor")
# stopwords.remove("not")

# print(stopwords) # see default stopwords


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def stop_word_removal(row):
    token = [token for token in row if token not in stopwords]
    token = filter(None, token)
    return token

# Assemble Embedding Vectors

The following functions are used to extract sent2vec embedding vectors for each review

In [9]:
import time
import sent2vec

s2v_model = sent2vec.Sent2vecModel()
start=time.time()
s2v_model.load_model('../input/sent2vec/wiki_unigrams.bin')
end = time.time()
print("Loading the sent2vec embedding took %d seconds"%(end-start))

Loading the sent2vec embedding took 10 seconds


In [10]:
def assemble_embedding_vectors(data):
    out = None
    for item in data:
        vec = s2v_model.embed_sentence(" ".join(str(i) for i in item if i))
        if vec is not None:
            if out is not None:
                out = np.concatenate((out,vec),axis=0)
            else:
                out = vec                                            
        else:
            pass
        
        
    return out

# Putting It All Together To Assemble Dataset

Now, putting all the preprocessing steps together we assemble our dataset...

In [11]:
import os
import numpy as np

# shuffle raw data first
def unison_shuffle_data(data, header):
    p = np.random.permutation(len(header))
    data = data[p]
    header = np.asarray(header)[p]
    return data, header

def load_data(path):
    data, sentiments = [], []
    for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in os.listdir(folder):
            with open(os.path.join(folder, name), 'r') as reader:
                  text = reader.read()
            text = tokenize(text)
            text = stop_word_removal(text)
            text = remove_reg_expressions(text)
            data.append(text)
            sentiments.append(sentiment)
    data_np = np.array(data)
    data, sentiments = unison_shuffle_data(data_np, sentiments)
    
    return data, sentiments

train_path = os.path.join('aclImdb', 'train')
test_path = os.path.join('aclImdb', 'test')
raw_data, raw_header = load_data(train_path)

print(raw_data.shape)
print(len(raw_header))

(25000,)
25000


In [12]:
# Subsample required number of samples
random_indices = np.random.choice(range(len(raw_header)),size=(Nsamp*2,),replace=False)
data_train = raw_data[random_indices]
header = raw_header[random_indices]

del raw_data, raw_header # huge and no longer needed, get rid of it

print("DEBUG::data_train::")
print(data_train)

DEBUG::data_train::
[list(['made', 'year', ' vertigo  ', 'equally', 'bewitching', 'movie ', 'though', 'much', 'lighter', 'vein ', 'it s', 'set', 'enchanted', 'new', 'york', 'winter ', 'kim', 'novak', 'witch', 'casts', 'spell', 'james', 'stewart ', 'gets', 'caught', 'instead ', 'the', 'interesting', 'sidelight', 'novak s', 'rival', 'played', 'janice', 'rule ', 'originated', 'part', 'madge', ' picnic ', 'broadway', ' the', 'part', 'novak', 'would', 'make', 'famous', 'film  '])
 list(['just', 'kidding  br', '   br', '  seeking', 'greener', 'pastures', 'form', 'hustling', 'new', 'york', 'city ', 'jon', 'voight', 'young', 'optimist', 'cowboy', ' almost', 'forest', 'gump like ', 'joe', 'buck', 'texas ', 'it', 'take', 'long', 'big', 'apple', 'mercilessly', 'swallow', 'ambitions', 'whole', 'soon', 'joe', 'target', 'coldness', 'new', 'yorkers', 'cons', 'street thugs ', 'given', 'pure', 'heart ', 'takes', 'pity', 'one', 'thugs ', 'ratso', 'rizzo', ' dustin', 'hoffman ', 'later', 'moves', 'wreck'

Display sentiments and their frequencies in the dataset, to ensure it is roughly balanced between classes

In [13]:
unique_elements, counts_elements = np.unique(header, return_counts=True)
print("Sentiments and their frequencies:")
print(unique_elements)
print(counts_elements)

Sentiments and their frequencies:
[0 1]
[1033  967]


**Featurize and Create Labels**

In [14]:
EmbeddingVectors = assemble_embedding_vectors(data_train)
print(EmbeddingVectors)

[[-0.1865912   0.00296916 -0.08566295 ...  0.0182172  -0.01269061
   0.21264666]
 [-0.07407703 -0.08794717  0.0178676  ...  0.04186484 -0.08544751
   0.03688974]
 [ 0.01704564 -0.10370536 -0.0200744  ...  0.03406874 -0.10187364
   0.17778794]
 ...
 [ 0.12999517 -0.05254492  0.18705492 ...  0.0082718  -0.03404788
   0.06047222]
 [-0.07268659 -0.00444807 -0.02116374 ...  0.04578894  0.00873205
   0.0256112 ]
 [ 0.02657777 -0.03309947 -0.12364911 ...  0.07154018 -0.06795725
   0.10368899]]


In [15]:
data = EmbeddingVectors

idx = int(0.7*data.shape[0])

# 70% of data for training
train_x = data[:idx,:]
train_y = np.array(header[:idx])
# # remaining 30% for testing
test_x = data[idx:,:]
test_y = np.array(header[idx:]) 

print("train_x/train_y list details, to make sure it is of the right form:")
print(len(train_x))
print(train_x)
print(train_y[:5])
print(len(train_y))

train_x/train_y list details, to make sure it is of the right form:
1400
[[-0.1865912   0.00296916 -0.08566295 ...  0.0182172  -0.01269061
   0.21264666]
 [-0.07407703 -0.08794717  0.0178676  ...  0.04186484 -0.08544751
   0.03688974]
 [ 0.01704564 -0.10370536 -0.0200744  ...  0.03406874 -0.10187364
   0.17778794]
 ...
 [ 0.14878118 -0.05725068  0.1905301  ...  0.12106173  0.10261593
  -0.00295256]
 [ 0.13707681 -0.08506337  0.17267852 ...  0.01870527  0.07860093
   0.07494058]
 [ 0.02448745  0.09204672 -0.02318908 ...  0.11524606  0.03332767
   0.06049296]]
[1 1 1 1 1]
1400


# Train Shallow Model for IMDB Reviews

In [16]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout

input_shape = (len(train_x[0]),)
sent2vec_vectors = Input(shape=input_shape)
dense = Dense(512, activation='relu')(sent2vec_vectors)
dense = Dropout(0.1)(dense)
output = Dense(1, activation='sigmoid')(dense)
model = Model(inputs=sent2vec_vectors, outputs=output)

Using TensorFlow backend.


In [17]:
model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
history = model.fit(train_x, train_y, validation_data=(test_x, test_y), batch_size=32,
                    nb_epoch=10, shuffle=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 1400 samples, validate on 600 samples
Epoch 1/10
1400/1400 [==============================] - 1s 415us/step - loss: 0.5649 - accuracy: 0.7050 - val_loss: 0.4669 - val_accuracy: 0.7767
Epoch 2/10
1400/1400 [==============================] - 0s 172us/step - loss: 0.4055 - accuracy: 0.8186 - val_loss: 0.4193 - val_accuracy: 0.8050
Epoch 3/10
1400/1400 [==============================] - 0s 170us/step - loss: 0.3576 - accuracy: 0.8364 - val_loss: 0.4070 - val_accuracy: 0.8000
Epoch 4/10
1400/1400 [==============================] - 0s 153us/step - loss: 0.3258 - accuracy: 0.8636 - val_loss: 0.4096 - val_accuracy: 0.8167
Epoch 5/10
1400/1400 [==============================] - 0s 158us/step - loss: 0.2856 - accuracy: 0.8771 - val_loss: 0.4106 - val_accuracy: 0.8183
Epoch 6/10
1400/1400 [==============================] - 0s 146us/step - loss: 0.2634 - accuracy: 0.8943 - val_loss: 0.4150 - val_accuracy: 0.8150
Epoch 7/10
1400/1400 [==============================] - 0s 151us/step - loss:

# Test Trained Model on Book Reviews from MDSD

In [18]:
!ls ../input/multi-domain-sentiment-dataset-books-and-dvds/

books.negative.review  dvd.negative.review
books.positive.review  dvd.positive.review


In [19]:
def parse_MDSD(data):
    out_lst = []
    for i in range(len(data)):
        txt = ""
        if(data[i]=="<review_text>\n"):
            j=i
            while(data[j]!="</review_text>\n"):
                txt = txt+data[j]
                j = j+1
            text = tokenize(txt)
            text = stop_word_removal(text)
            text = remove_reg_expressions(text)
            out_lst.append(text)
            
            #print(txt)
            #print(text)
            
    return out_lst

with open ("../input/multi-domain-sentiment-dataset-books-and-dvds/books.negative.review", "r", encoding="latin1") as myfile:
    data=myfile.readlines()
neg_books = parse_MDSD(data)
len(neg_books)

with open ("../input/multi-domain-sentiment-dataset-books-and-dvds/books.positive.review", "r", encoding="latin1") as myfile:
    data=myfile.readlines()
pos_books = parse_MDSD(data)
len(pos_books)

#print(neg_books)
#print(pos_books)

header = [0]*len(neg_books)
header.extend([1]*len(pos_books))
neg_books.extend(pos_books)
MDSD_data = np.array(neg_books)

data, sentiments = unison_shuffle_data(np.array(MDSD_data), header)

len(sentiments)

2000

**Try using the IMDB classifier directly on book review data...**

In [20]:
EmbeddingVectors = assemble_embedding_vectors(data)
print(EmbeddingVectors)
sentiments = np.asarray(sentiments)

[[-0.02857956 -0.19712123 -0.03367234 ...  0.16281778 -0.05813418
   0.03944178]
 [-0.03060782 -0.07786036  0.02195755 ...  0.07094602 -0.13154835
  -0.0274305 ]
 [-0.00442837 -0.11807001  0.07095549 ... -0.04005149 -0.12931705
   0.17400824]
 ...
 [-0.09499011  0.02815587  0.01926375 ... -0.05984141 -0.09697013
   0.20014554]
 [-0.13298799 -0.12199261 -0.07488126 ...  0.08148796 -0.14331028
   0.12039624]
 [-0.11523205 -0.09352691  0.04332658 ...  0.12436987 -0.1109891
   0.03355481]]


In [21]:

print(model.evaluate(x=EmbeddingVectors,y=sentiments)) # evaluate IMDB classifier on books directly
print(model.metrics_names)

2000/2000 [==============================] - 0s 38us/step
[0.6671734366416932, 0.7440000176429749]
['loss', 'accuracy']


# Adaptation of Book Review Domain via Autoencoder

In [22]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

encoding_dim = 30 # chosen experimentally

input_shape = (len(train_x[0]),)
sent2vec_vectors = Input(shape=input_shape)
encoder = Dense(encoding_dim)(sent2vec_vectors)
dropout = Dropout(0.1)(encoder)
decoder = Dense(encoding_dim)(dropout)
dropout = Dropout(0.1)(decoder)
output = Dense(len(train_x[0]))(dropout)
autoencoder = Model(inputs=sent2vec_vectors, outputs=output)

In [23]:
autoencoder.compile(optimizer='adam',loss='mse',metrics=["mse","mae"])
autoencoder.fit(train_x,train_x,validation_data=(test_x, test_x), batch_size=32,
                    nb_epoch=50, shuffle=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 1400 samples, validate on 600 samples
Epoch 1/50
1400/1400 [==============================] - 0s 263us/step - loss: 0.0144 - mse: 0.0144 - mae: 0.0932 - val_loss: 0.0092 - val_mse: 0.0092 - val_mae: 0.0750
Epoch 2/50
1400/1400 [==============================] - 0s 77us/step - loss: 0.0090 - mse: 0.0090 - mae: 0.0743 - val_loss: 0.0081 - val_mse: 0.0081 - val_mae: 0.0707
Epoch 3/50
1400/1400 [==============================] - 0s 76us/step - loss: 0.0083 - mse: 0.0083 - mae: 0.0714 - val_loss: 0.0078 - val_mse: 0.0078 - val_mae: 0.0692
Epoch 4/50
1400/1400 [==============================] - 0s 77us/step - loss: 0.0078 - mse: 0.0078 - mae: 0.0695 - val_loss: 0.0074 - val_mse: 0.0074 - val_mae: 0.0675
Epoch 5/50
1400/1400 [==============================] - 0s 76us/step - loss: 0.0075 - mse: 0.0075 - mae: 0.0679 - val_loss: 0.0072 - val_mse: 0.0072 - val_mae: 0.0662
Epoch 6/50
1400/1400 [==============================] - 0s 78us/step - loss: 0.0072 - mse: 0.0072 - mae: 0.0667 - val

In [24]:
# transform EmbeddingVectors and sentiments with autoencoder.predict and then evaluate IDMB model again

EmbeddingVectorsProjected = autoencoder.predict(EmbeddingVectors)

print(EmbeddingVectorsProjected.shape)

print(model.evaluate(x=EmbeddingVectorsProjected,y=sentiments))
print(model.metrics_names)

(2000, 600)
2000/2000 [==============================] - 0s 38us/step
[0.5876209111213684, 0.75]
['loss', 'accuracy']


In [25]:
from IPython.display import HTML
def create_download_link(title = "Download file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

#create_download_link(filename='GBMimportances.svg')

In [26]:
!rm -rf aclImdb
!rm aclImdb_v1.tar.gz